In [ ]:
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

from IPython.display import display

# remove warnings
import warnings
warnings.filterwarnings('ignore')

# Load and pick Datas

In [ ]:
# 读取训练集和测试集为 DataFrame
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

print train_data.shape
display(train_data.head(1))
# display(train_data.info())

print test_data.shape
display(test_data.head(1))
# display(test_data.info())
train_length = train_data.shape[0]

# Filling up missing values

In [ ]:
def conbine_train_test_data():
    train_data2 = train_data.drop('SalePrice', axis=1)
    return pd.concat([train_data2, test_data])

conbined_data = conbine_train_test_data()
conbined_data.shape

In [ ]:
# 存在缺失数据的列名
has_null_columns = conbined_data.columns[conbined_data.isnull().any()].tolist()
# 每列包含多少缺失数据
conbined_data[has_null_columns].isnull().sum()

## Data Cleaning Plan
* `LotFrontage`: Linear feet of street connected to property. Impute mean, or maybe see if there's a correlation with `LotArea` (like square root?).

* `Alley`: Type of alley access to property -> Many missing values, I would presume that these properties just don't have an alley access.

* `MasVnrType/MasVnrArea` -> both have 8 values missing, I presume they are the same ones. Either set as "None"/0 or use most frequent value/median.

* `Bsmt... Variables`: A  number of variables in connection with the basement. About the same number of missing values. However, there are two basement-related variables without missing values "BsmtFinSF1" and "BsmtFinSF2" - look at those and then decide what to do with the missing values.

* `Electrical`: Just one missing value - here just impute most frequent one.

* `FireplaceQu`: I assume the properties with missing values just don't have a fireplace. There's also the variable Fireplaces (without missing values) - check this and then decide.

* `Garage ... Variables`: 81 missing in these columns. However, there are some Garage-related variables without missing values: GarageCars, GarageArea - check these and then decide.

* `PoolQC` - probably no pool - but check against PoolArea (which has no missing values).

* `Fence`: Many missing values - probably no fence, just impute 'None'

* `MiscFeature`: Assuming none - probably no special features, just impute 'None'

In [ ]:
def fill_missing_conbined_data(column, value):
    conbined_data.loc[conbined_data[column].isnull(),column] = value

** LotFrontage/LotArea **

对于缺失的 LotFrontage，一般采用平均值进行填充，但考虑到可能与 LotArea 存在关系。

In [ ]:
# Compute pairwise correlation of columns, excluding NA/null values
conbined_data['LotFrontage'].corr(conbined_data['LotArea'])

LotArea: Lot size in square feet。LotArea 平方英尺为单位。考虑单位同一将 LotArea 进行 `sqrt()`

In [ ]:
conbined_data['SqrtLotArea'] = np.sqrt(conbined_data['LotArea'])
conbined_data['LotFrontage'].corr(conbined_data['SqrtLotArea'])

相关系数为 0.64765803986178294，说明 LotFrontage 与 LotArea 之间存在一定的正相关。下面对LotFrontage 与 LotArea 进行拟合。 

In [ ]:
# 绘制 LotFrontage 与 LotArea 原始数据
lot_datas = conbined_data[['LotFrontage','SqrtLotArea']].dropna()
plt.figure(figsize=(5,4))
plt.scatter(lot_datas['SqrtLotArea'], lot_datas['LotFrontage'])
plt.xlabel("SqrtLotArea") 
plt.ylabel("LotFrontage") 

In [ ]:
from sklearn.linear_model import LinearRegression

# Create linear regression object
regr = LinearRegression()

In [ ]:
lotFrontage = lot_datas.iloc[:,0].values
sqrtLotArea = lot_datas.iloc[:,1].values

length = lotFrontage.shape[0]
lotFrontage = lotFrontage.reshape(length, 1)
sqrtLotArea = sqrtLotArea.reshape(length, 1)

# Train the model using the training sets
regr.fit(sqrtLotArea, lotFrontage)

In [ ]:
# The coefficients
print('Coefficients: \n', regr.coef_)

In [ ]:
plt.scatter(sqrtLotArea, lotFrontage,  color='blue')
plt.plot(sqrtLotArea, regr.predict(sqrtLotArea), color='red',
         linewidth=3)

In [ ]:
def fill_missing_lotfrontage(sqrt_lotarea):
    """
    填充缺失的 LotFrontage 数据
    """
    return 0.6430147 * sqrt_lotarea

column = conbined_data['LotFrontage'].isnull()
conbined_data['LotFrontage'][column] = conbined_data['SqrtLotArea'][column].map(fill_missing_lotfrontage)

In [ ]:
conbined_data[conbined_data['LotFrontage'].isnull()]

In [ ]:
conbined_data.drop('SqrtLotArea', axis=1, inplace=True)
conbined_data.shape

** Alley **

In [ ]:
conbined_data.shape, conbined_data[conbined_data['Alley'].isnull()].shape

2919 条数据缺失 2721 条，缺失数据过多(93.2%),将缺失数据填充为 NA（NA->No alley access）。

In [ ]:
fill_missing_conbined_data('Alley', 'NA')
conbined_data[conbined_data['Alley'].isnull()]

** MasVnrType / MasVnrArea **

方形单板砌体类型/面积，将缺失数据填充为出现次数最多的类型

In [ ]:
conbined_data['MasVnrType'].fillna('None', inplace=True)
conbined_data['MasVnrArea'].fillna(0, inplace=True)

In [ ]:
conbined_data['MasVnrType'].isnull().sum(), conbined_data['MasVnrArea'].isnull().sum()

** BsmtQual / BsmtCond / BsmtExposure / BsmtFinType1 / BsmtFinType2 **

缺失 37 / 38 条数据。

In [ ]:
basement_cols=['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtFinSF1','BsmtFinSF2']

In [ ]:
# 对于缺失的字符串类型的数据填充为 NA，表示 No Basement
for column in basement_cols:
    if 'FinSF'not in column:
        # NA	No Basement
        fill_missing_conbined_data(column, 'NA')

In [ ]:
# 对于数值型的 BsmtFinSF1 和 BsmtFinSF2
fill_missing_conbined_data('BsmtFinSF1', 0)
fill_missing_conbined_data('BsmtFinSF2', 0)

In [ ]:
conbined_data[basement_cols].isnull().sum()

** Electrical **

缺失一条数据,填充为出现次数最多的。

In [ ]:
fill_missing_conbined_data('Electrical', conbined_data['Electrical'].max())

** FireplaceQu **

缺失 1420 条数据
* Fireplaces: Number of fireplaces
* FireplaceQu: Fireplace quality

所以 FireplaceQu 与 Fireplaces 有关系

In [ ]:
conbined_data['FireplaceQu'].value_counts()
conbined_data[['FireplaceQu', 'Fireplaces']].groupby(['FireplaceQu']).median()
sns.countplot(conbined_data['FireplaceQu'])

In [ ]:
# 对于 Fireplaces 为 0 的，FireplaceQu 设置为 NA，表示 No Fireplace，此即缺失的 1420 条数据的情况
fill_missing_conbined_data('FireplaceQu', 'NA')

** PoolQC **

PoolQC 缺失 2909 条数据，是否与 PoolArea 有关。

In [ ]:
fill_missing_conbined_data('PoolQC', 'NA')

** MiscFeature **

In [ ]:
fill_missing_conbined_data('MiscFeature', 'NA')

** Fence **

In [ ]:
fill_missing_conbined_data('Fence', 'NA')

** Garages **

In [ ]:
garage_cols=['GarageType','GarageQual','GarageCond','GarageYrBlt','GarageFinish','GarageCars','GarageArea']
conbined_data[garage_cols][conbined_data['GarageType'].isnull()==True].head(1)

In [ ]:
# 对于缺失的字符串类型的数据填充为 NA，表示 No Garage
for column in garage_cols:
    if column != 'GarageCars' and column != 'GarageArea':
        # NA	No Basement
        fill_missing_conbined_data(column, 'NA')
    else:
        fill_missing_conbined_data(column, 0)

** MSZoning **

In [ ]:
sns.countplot(conbined_data['MSZoning'])

In [ ]:
conbined_data['MSZoning'].max()

In [ ]:
fill_missing_conbined_data('MSZoning', conbined_data['MSZoning'].max())

** Utilities **

In [ ]:
fill_missing_conbined_data('Utilities', conbined_data['Utilities'].max())

** Exterior1st / Exterior2nd**

In [ ]:
fill_missing_conbined_data('Exterior1st', conbined_data['Exterior1st'].max())

In [ ]:
fill_missing_conbined_data('Exterior2nd', conbined_data['Exterior2nd'].max())

** BsmtUnfSF / TotalBsmtSF **

BsmtUnfSF: Unfinished square feet of basement area<br/>
BsmtFinSF2: Type 2 finished square feet<br/>
BsmtFinSF1: Type 1 finished square feet<br/>
TotalBsmtSF: Total square feet of basement area

测试发现：`TotalBsmtSF = BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF`

In [ ]:
regr = LinearRegression()
bsmt_data = conbined_data[['BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF']].dropna()

In [ ]:
bsmt_x = bsmt_data.iloc[:,0:-1].values
bsmt_y = bsmt_data.iloc[:,-1].values

length = bsmt_x.shape[0]
bsmt_y = bsmt_y.reshape(length, 1)

# Train the model using the training sets
regr.fit(bsmt_x, bsmt_y)

In [ ]:
# The coefficients
print('Coefficients: \n', regr.coef_)

In [ ]:
conbined_data[['BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF']][conbined_data['BsmtUnfSF'].isnull()]

In [ ]:
# 可以发现，对于缺失的 TotalBsmtSF 和 BsmtUnfSF 填充为0
fill_missing_conbined_data('BsmtUnfSF', 0)
fill_missing_conbined_data('TotalBsmtSF', 0)

** BsmtFullBath / BsmtHalfBath **

BsmtFullBath: Basement full bathrooms<br/>
BsmtHalfBath: Basement half bathrooms<br/>
FullBath: Full bathrooms above grade<br/>
HalfBath: Half baths above grade

In [ ]:
conbined_data['FullBath'].corr(conbined_data['BsmtFullBath'])

In [ ]:
fill_missing_conbined_data('BsmtFullBath', 0)
fill_missing_conbined_data('BsmtHalfBath', 0)

** KitchenQual **

KitchenAbvGr: Kitchens above grade<br/>
KitchenQual: Kitchen quality

In [ ]:
print conbined_data['KitchenQual'].max()
conbined_data[['KitchenAbvGr', 'KitchenQual']].groupby('KitchenQual').mean()

In [ ]:
# 差别并不大，填充为1
fill_missing_conbined_data('KitchenQual', conbined_data['KitchenQual'].max())

** SaleType / Functional **

In [ ]:
fill_missing_conbined_data('SaleType', conbined_data['SaleType'].max())
fill_missing_conbined_data('Functional', conbined_data['Functional'].max())

完成了缺失数据的填充。

In [ ]:
# 存在缺失数据的列名
has_null_columns = conbined_data.columns[conbined_data.isnull().any()].tolist()
# 每列包含多少缺失数据
conbined_data[has_null_columns].isnull().sum()

# Feature Engineering

** 时间属性（YearBuilt、GarageYrBlt）**

In [ ]:
built_year_data = conbined_data[['YearBuilt', 'GarageYrBlt']][conbined_data['GarageYrBlt'] != 'NA']

In [ ]:
built_year_data['GarageYrBlt'] = built_year_data['GarageYrBlt'].map(lambda g : int(g))
built_year_data['GarageYrBlt'].corr(built_year_data['YearBuilt'])

可以发现 YearBuilt、GarageYrBlt 之间存在很高的相关性。

In [ ]:
built_year_data.head(1)

In [ ]:
regr = LinearRegression()
garage_year = built_year_data.loc[:,'GarageYrBlt'].values
built_year = built_year_data.loc[:,'YearBuilt'].values

length = garage_year.shape[0]
garage_year = garage_year.reshape(length, 1)
built_year = built_year.reshape(length, 1)

# Train the model using the training sets
regr.fit(built_year, garage_year)
plt.scatter(built_year, garage_year,  color='blue')
plt.plot(built_year, regr.predict(built_year), color='red',
         linewidth=3)

# The coefficients
print('Coefficients: \n', regr.coef_)

In [ ]:
# 对于 NA 的 GarageYrBlt，进行填充
conbined_data['GarageYrBlt'] = conbined_data.apply(lambda row : regr.predict(row['YearBuilt'])
                                                      if row['GarageYrBlt'] == 'NA' else row['GarageYrBlt'],
                                                       axis=1)

conbined_data['GarageYrBlt'] = conbined_data['GarageYrBlt'].map(lambda g : float(g))

In [ ]:
conbined_data['GarageYrBlt'].head()

对于 MSSubClass 的数值仅仅代表 the type of dwelling，所以将其编码。

In [ ]:
np.array(set(conbined_data['MSSubClass']))

In [ ]:
mssubclass_dict = {
    20: 'SC20',
    30: 'SC30',
    40: 'SC40',
    45: 'SC45',
    50: 'SC50',
    60: 'SC60',
    70: 'SC70',
    75: 'SC75',
    80: 'SC80',
    85: 'SC85',
    90: 'SC90',
    120: 'SC120',
    150: 'SC150',
    160: 'SC160',
    180: 'SC180',
    190: 'SC190',
}
conbined_data['MSSubClass'] = conbined_data['MSSubClass'].replace(mssubclass_dict)

In [ ]:
np.array(set(conbined_data['MSSubClass']))

对于那些存在大小特质的属性进行编码。

In [ ]:
conbined_data = conbined_data.replace(
    {'Street': {'Grvl': 0, 'Pave': 1},
     'Alley': {'NA':0, 'Grvl': 1, 'Pave': 2},
     'LotShape': {'IR1':2, 'IR2':1, 'IR3':0, 'Reg':3},
     'LandContour': {'Low':0, 'HLS':1, 'Bnk':2, 'Lvl':3},
     'Utilities': {'AllPub':3, 'NoSeWa': 1, 'NoSewr': 2, 'ELO': 0},
     'LandSlope': {'Sev':0, 'Gtl':2, 'Mod':1},
     'ExterQual': {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1,'Po': 0},
     'ExterCond': {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1,'Po': 0},
     'BsmtExposure': {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1,'NA': 0},
     'BsmtQual': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2,'Po': 1,'NA': 0},
     'BsmtCond': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2,'Po': 1,'NA': 0},
     'BsmtFinType1': {'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
     'BsmtFinType2': {'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
     'HeatingQC': {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1,'Po': 0},
     'CentralAir': {'N':0, 'Y':1},
     'Electrical': {'Mix':0, 'FuseP':1, 'FuseF':2, 'FuseA':3, 'SBrkr':5},
     'KitchenQual': {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0},
     'Functional': {'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,'Maj2':2,'Sev':1,'Sal': 0},
     'FireplaceQu': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0},
     'GarageQual': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0},
     'GarageCond': {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0},
     'PoolQC': {'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'NA': 0},
     'Fence': {'GdPrv': 4, 'GdWo': 3, 'MnPrv': 2, 'MnWw': 1, 'NA': 0},
    })

Neighborhood 属性表示的是附近的地名，可将其转为经纬度。

In [ ]:
# 纬度
conbined_data["latitude"] = conbined_data.Neighborhood.replace(
                                               {'Blmngtn' : 42.062806,
                                                'Blueste' : 42.009408,
                                                'BrDale' : 42.052500,
                                                'BrkSide': 42.033590,
                                                'ClearCr': 42.025425,
                                                'CollgCr': 42.021051,
                                                'Crawfor': 42.025949,
                                                'Edwards': 42.022800,
                                                'Gilbert': 42.027885,
                                                'GrnHill': 42.000854,
                                                'IDOTRR' : 42.019208,
                                                'Landmrk': 42.044777,
                                                'MeadowV': 41.991866,
                                                'Mitchel': 42.031307,
                                                'NAmes'  : 42.042966,
                                                'NoRidge': 42.050307,
                                                'NPkVill': 42.050207,
                                                'NridgHt': 42.060356,
                                                'NWAmes' : 42.051321,
                                                'OldTown': 42.028863,
                                                'SWISU'  : 42.017578,
                                                'Sawyer' : 42.033611,
                                                'SawyerW': 42.035540,
                                                'Somerst': 42.052191,
                                                'StoneBr': 42.060752,
                                                'Timber' : 41.998132,
                                                'Veenker': 42.040106})
# 经度
conbined_data["longitude"] = conbined_data.Neighborhood.replace(
                                               {'Blmngtn' : -93.639963,
                                                'Blueste' : -93.645543,
                                                'BrDale' : -93.628821,
                                                'BrkSide': -93.627552,
                                                'ClearCr': -93.675741,
                                                'CollgCr': -93.685643,
                                                'Crawfor': -93.620215,
                                                'Edwards': -93.663040,
                                                'Gilbert': -93.615692,
                                                'GrnHill': -93.643377,
                                                'IDOTRR' : -93.623401,
                                                'Landmrk': -93.646239,
                                                'MeadowV': -93.602441,
                                                'Mitchel': -93.626967,
                                                'NAmes'  : -93.613556,
                                                'NoRidge': -93.656045,
                                                'NPkVill': -93.625827,
                                                'NridgHt': -93.657107,
                                                'NWAmes' : -93.633798,
                                                'OldTown': -93.615497,
                                                'SWISU'  : -93.651283,
                                                'Sawyer' : -93.669348,
                                                'SawyerW': -93.685131,
                                                'Somerst': -93.643479,
                                                'StoneBr': -93.628955,
                                                'Timber' : -93.648335,
                                                'Veenker': -93.657032})

# Ames city 的经纬度：经度-93.63191310000002，纬度42.0307812，计算距离
conbined_data["NeighborDistance"] = np.sqrt(np.power((conbined_data["longitude"] - (-93.63191310000002)),2) + \
                                    np.power((conbined_data["latitude"] - 42.0307812),2))

display(conbined_data["NeighborDistance"].head())

# Bin by neighborhood (a little arbitrarily). Values were computed by: 
# train_df["SalePrice"].groupby(train_df["Neighborhood"]).median().sort_values()
neighborhood_map = {
    "MeadowV" : 0,  #  88000
    "IDOTRR" : 1,   # 103000
    "BrDale" : 1,   # 106000
    "OldTown" : 1,  # 119000
    "Edwards" : 1,  # 119500
    "BrkSide" : 1,  # 124300
    "Sawyer" : 1,   # 135000
    "Blueste" : 1,  # 137500
    "SWISU" : 2,    # 139500
    "NAmes" : 2,    # 140000
    "NPkVill" : 2,  # 146000
    "Mitchel" : 2,  # 153500
    "SawyerW" : 2,  # 179900
    "Gilbert" : 2,  # 181000
    "NWAmes" : 2,   # 182900
    "Blmngtn" : 2,  # 191000
    "CollgCr" : 2,  # 197200
    "ClearCr" : 3,  # 200250
    "Crawfor" : 3,  # 200624
    "Veenker" : 3,  # 218000
    "Somerst" : 3,  # 225500
    "Timber" : 3,   # 228475
    "StoneBr" : 4,  # 278000
    "NoRidge" : 4,  # 290000
    "NridgHt" : 4,  # 315000
}

conbined_data["NeighborhoodLevelBin"] = conbined_data["Neighborhood"].map(neighborhood_map)

# 去掉字符串类型的 Neighborhood
conbined_data.drop('Neighborhood', axis=1, inplace=True)
conbined_data.drop('latitude', axis=1, inplace=True)
conbined_data.drop('longitude', axis=1, inplace=True)

YearBuilt建造时间，YearRemodAdd修建时间，确定是否翻新改造过

In [ ]:
conbined_data['RemodYears'] = conbined_data['YearRemodAdd'] - conbined_data['YearBuilt']

In [ ]:
conbined_data['GarageBltYears'] = conbined_data['GarageYrBlt'] - conbined_data['YearBuilt']

In [ ]:
conbined_data['GarageBltYears'].head()

In [ ]:
# 距离现在建造的时间间隔
conbined_data['Now_YearBuilt'] = 2017 - conbined_data['YearBuilt']
conbined_data['Now_YearRemodAdd'] = 2017 - conbined_data['YearRemodAdd']
conbined_data['Now_GarageYrBlt'] = 2017 - conbined_data['GarageYrBlt']

conbined_data.drop('YearBuilt', axis=1, inplace=True)
conbined_data.drop('YearRemodAdd', axis=1, inplace=True)
conbined_data.drop('GarageYrBlt', axis=1, inplace=True)

In [ ]:
conbined_data[['Now_YearBuilt','Now_YearRemodAdd','Now_GarageYrBlt']].head()

房子卖的月份为数值类型，将其转为字符串类型

In [ ]:
conbined_data['MoSold'] = conbined_data['MoSold'].map(lambda m : str(m))

In [ ]:
conbined_data['MoSold'].head()

# Create new features

Ref: [juliencs : A study on Regression applied to the Ames dataset
](https://www.kaggle.com/juliencs/house-prices-advanced-regression-techniques/a-study-on-regression-applied-to-the-ames-dataset)
- Simplifications of existing features
- Combinations of existing features
- Polynomials on the top 10 existing features

In [ ]:
# 1.Simplifications of existing features
conbined_data["SimplOverallQual"] = conbined_data.OverallQual.replace(
                                                            {1 : 1, 2 : 1, 3 : 1, # bad
                                                             4 : 2, 5 : 2, 6 : 2, # average
                                                             7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                            })
conbined_data["SimplOverallCond"] = conbined_data.OverallCond.replace(
                                                            {1 : 1, 2 : 1, 3 : 1, # bad
                                                             4 : 2, 5 : 2, 6 : 2, # average
                                                             7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                             })

In [ ]:
# 2.Combinations of existing features

# Overall quality of the house
conbined_data["OverallGrade"] = conbined_data["OverallQual"] * conbined_data["OverallCond"]
# Overall quality of the garage
conbined_data["GarageGrade"] = conbined_data["GarageQual"] * conbined_data["GarageCond"]
# Overall quality of the exterior
conbined_data["ExterGrade"] = conbined_data["ExterQual"] * conbined_data["ExterCond"]
# Overall kitchen score
conbined_data["KitchenScore"] = conbined_data["KitchenAbvGr"] * conbined_data["KitchenQual"]
# Overall fireplace score
conbined_data["FireplaceScore"] = conbined_data["Fireplaces"] * conbined_data["FireplaceQu"]
# Overall garage score
conbined_data["GarageScore"] = conbined_data["GarageArea"] * conbined_data["GarageQual"]
# Overall pool score
conbined_data["PoolScore"] = conbined_data["PoolArea"] * conbined_data["PoolQC"]

# Total number of bathrooms
conbined_data["TotalBath"] = conbined_data["BsmtFullBath"] + (0.5 * conbined_data["BsmtHalfBath"]) + \
conbined_data["FullBath"] + (0.5 * conbined_data["HalfBath"])

# Total yard area in square feet
conbined_data["TotalPorchSF"] = conbined_data["OpenPorchSF"] + conbined_data["EnclosedPorch"] +\
                                conbined_data["3SsnPorch"] + conbined_data["ScreenPorch"]
# Total SF for house (living, basement, porch, pool)
conbined_data["AllSF"] = conbined_data["GrLivArea"] + conbined_data["TotalBsmtSF"] + \
                         conbined_data["TotalPorchSF"] + conbined_data["WoodDeckSF"] + \
                         conbined_data["PoolArea"]

# Has masonry veneer or not
conbined_data["HasMasVnr"] = conbined_data.MasVnrType.replace(
    {"BrkCmn" : 1, "BrkFace" : 1, "CBlock" : 1, "Stone" : 1, "None" : 0})
# House completed before sale or not
conbined_data["BoughtOffPlan"] = conbined_data.SaleCondition.replace(
    {"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0, "Family" : 0, "Normal" : 0, "Partial" : 1})

In [ ]:
# 3.Polynomials on the top 10 existing features
train_data_new = conbined_data.iloc[:train_length,:]
# 添加价格
train_data_new['SalePrice'] = train_data['SalePrice']
train_data_new.shape

In [ ]:
print("Find most important features relative to target")
corr = train_data_new.corr()
corr.sort_values(["SalePrice"], ascending = False, inplace = True)
corr_saleprice = corr.SalePrice
corr_saleprice.head(11)

del train_data_new

In [ ]:
# 3.Polynomials on the top 10 existing features
top_n = 10
poly_columns = corr_saleprice.index.values[1:top_n+1]
for column in poly_columns:
    conbined_data[column+'-s2'] = conbined_data[column] ** 2
    conbined_data[column+'-s3'] = conbined_data[column] ** 2
    conbined_data[column+'-sq'] = np.sqrt(conbined_data[column])

** Feature Scale / Skew **

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# remove warnings
import warnings
warnings.filterwarnings('ignore')

num_column_names = conbined_data.select_dtypes(exclude=['object']).columns.values
num_column_names = num_column_names[1:]
display(num_column_names)

display(conbined_data[num_column_names].head())
conbined_data[num_column_names] = conbined_data[num_column_names].apply(
                           lambda x: MinMaxScaler().fit_transform(x))

display(conbined_data[num_column_names].head())

In [ ]:
sns.kdeplot(conbined_data['Now_YearBuilt'])

![](http://itknowledgeexchange.techtarget.com/writing-for-business/files/2012/12/skewness-300x247.png)

In [ ]:
# skewness 转换
conbined_data[num_column_names] = np.log1p(conbined_data[num_column_names])

** Dummy Encoding **

In [ ]:
conbined_data = pd.get_dummies(conbined_data)

In [ ]:
conbined_data.head(2)

In [ ]:
print("New number of features : " + str(conbined_data.shape[1] - 1))

In [ ]:
train_X = conbined_data.iloc[:train_length, 1:]
train_Y = train_data['SalePrice']
train_Id = conbined_data.iloc[:train_length, 0]

test_X = conbined_data.iloc[train_length:, 1:]
test_Id = conbined_data.iloc[train_length:, 0]

print("train_X : " + str(train_X.shape))
print("test_X : " + str(test_X.shape))
print("train_Y : " + str(train_Y.shape))

In [ ]:
# 价格分布
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(10,5))
axis1.hist(train_Y)
# Transform skewed numeric features using log(p+1) transformation making them more normal
train_Y = np.log1p(train_Y)
axis2.hist(train_Y)

# Save New Data

In [ ]:
save_train = train_X.copy()
save_test = test_X.copy()
save_train['Id'] = train_Id
save_train['SalePrice'] = train_Y
save_test['Id'] = test_Id
save_train.head()

save_train.to_csv("data/new_train.csv", index=False)
save_test.to_csv("data/new_test.csv", index=False)

# Modeling

** 训练集上交叉验证获取最佳模型参数 **

In [ ]:
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer

# Partition the dataset in train + validation sets
cross_train_X, cross_validate_X, cross_train_Y, cross_validate_Y = \
                train_test_split(train_X, train_Y, test_size = 0.3, random_state = 0)

print("cross_train_X : " + str(cross_train_X.shape))
print("cross_train_Y : " + str(cross_train_Y.shape))
print("cross_validate_X : " + str(cross_validate_X.shape))
print("cross_validate_Y : " + str(cross_validate_Y.shape))

In [ ]:
# Define error measure for official scoring : RMSE
scorer = make_scorer(mean_squared_error, greater_is_better = False)

def rmse_cv_train(model):
    rmse= np.sqrt(-cross_val_score(model, cross_train_X, cross_train_Y, scoring = scorer, cv = 10))
    return(rmse)

def rmse_cv_validate(model):
    rmse= np.sqrt(-cross_val_score(model, cross_validate_X, cross_validate_Y, scoring = scorer, cv = 10))
    return(rmse)

def predict_mean_squared_error(model, X, Y):
    predict_y = model.predict(X)
    return np.sqrt(np.power((predict_y - Y), 2))

Lasso Regression Model

In [ ]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(
            alphas = [0.0001, 0.0003, 0.001, 0.003, 0.009, 0.01, 0.03, 0.09, 0.1, 0.3, 1]
        ).fit(train_X, train_Y)

In [ ]:
lasso_error = predict_mean_squared_error(lasso, cross_train_X, cross_train_Y).mean()
lasso_error

In [ ]:
# coef = pd.Series(lasso.coef_, index = train_X.columns)

# # plotting feature importances!
# imp_coef = pd.concat([coef.sort_values().head(15),
#                      coef.sort_values().tail(15)])
# imp_coef.plot(kind = "barh")
# plt.title("Coefficients in the Lasso Model")

XGBRegressor Model

In [ ]:
import xgboost as xgb
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV

xgb_regr = xgb.XGBRegressor()

xgb_parameter_grid = {
                 'learning_rate' : [0.001, 0.025, 0.075, 0.1, 0.3, 0.6, 0.8, 1],
                 'n_estimators': [80,100,200,400,600,800,1000,1200,1400,1600,1800]
                 }
cross_validation = StratifiedKFold(train_Y, n_folds=5)
xgb_grid_search = GridSearchCV(xgb_regr,
                           param_grid=xgb_parameter_grid,
                           cv=cross_validation)

xgb_grid_search.fit(train_X, train_Y)

print('xgb Best score: {}'.format(xgb_grid_search.best_score_))
print('xgb Best parameters: {}'.format(xgb_grid_search.best_params_))

In [ ]:
xgb_error = predict_mean_squared_error(xgb_grid_search, cross_train_X, cross_train_Y).mean()
xgb_error

** 计算残差 residuals 验证模型的效果 **

In [ ]:
lasso_train_predict = lasso.predict(train_X)
xgb_train_predict = xgb_grid_search.predict(train_X)

residuals_lasso = lasso_train_predict - train_Y
residuals_xgb = xgb_train_predict - train_Y

In [ ]:
# 依据预测的 mean_squared_error 确定 voting 的权重
# xgb：0.02918017553888173，lasso：0.078234324232550001
weight_xgb = lasso_error / (xgb_error + lasso_error)
weight_lasso = xgb_error / (xgb_error + lasso_error)

voting_train_predict = weight_xgb * xgb_train_predict + weight_lasso * lasso_train_predict
residuals_voting = voting_train_predict - train_Y

In [ ]:
plt.plot(train_Id, residuals_lasso, c='blue')
plt.plot(train_Id, residuals_voting, c='yellow')
plt.plot(train_Id, residuals_xgb, c='red')

In [ ]:
print 'residuals_lasso = ', np.abs(residuals_lasso).mean()
print 'residuals_xgb = ', np.abs(residuals_xgb).mean()
print 'residuals_voting = ', np.abs(residuals_voting).mean()

可以看出，依据预测的平方误差设定的权重得到的加权结果的残差（黄色）要比单独的 lasso（蓝色）模型的残差要小很多，XGBRegressor 模型最好。

# Predict

In [ ]:
lasso_predictions = lasso.predict(test_X).astype(float)
lasso_predictions = np.power(np.e, lasso_predictions) - 1

print 'lasso predicts done.'

In [ ]:
xgb_predictions = xgb_grid_search.predict(test_X)
xgb_predictions = np.power(np.e, xgb_predictions) - 1

print 'xgb predicts done.'

In [ ]:
plt.plot(test_Id, lasso_predictions, c='red')
plt.plot(test_Id, xgb_predictions, c='blue')

voting_predictions = weight_xgb * xgb_predictions + weight_lasso * lasso_predictions
plt.plot(test_Id, voting_predictions, c='yellow')

In [ ]:
except_index = np.argwhere(lasso_predictions == lasso_predictions[lasso_predictions>600000])[0][0]
except_index

In [ ]:
submission = pd.DataFrame({
        "Id": test_Id,
        "SalePrice": xgb_predictions
    })

submission.to_csv("result/xgb_predictions_2_7.csv", index=False)

print "Done."

In [ ]:
submission = pd.DataFrame({
        "Id": test_Id,
        "SalePrice": voting_predictions
    })

submission.to_csv("result/weighted_voting_predictions_2_7.csv", index=False)

print "Done."